In [4]:
import torch 
import pandas as pd
import numpy as np
import random as rd
import networkx as nx
import matplotlib.pyplot as plt


In [2]:
movies = pd.read_csv("./Data/movies.csv",encoding="latin-1",sep="\t")
movies["movie_ID"] = np.arange(stop=movies.shape[0]) 

users = pd.read_csv("./Data/users.csv",encoding="latin-1",sep="\t")
users["user_ID"] = np.arange(stop=users.shape[0]) 

ratings = pd.read_csv("./Data/ratings.csv",encoding="latin-1",sep="\t")

edges = pd.merge(pd.merge(ratings,users[["user_id","user_ID"]],on="user_id"),movies[["movie_id","movie_ID"]],on="movie_id")

print(movies.head())
edges.head()

   movie_id                               title                        genres  \
0         1                    Toy Story (1995)   Animation|Children's|Comedy   
1         2                      Jumanji (1995)  Adventure|Children's|Fantasy   
2         3             Grumpier Old Men (1995)                Comedy|Romance   
3         4            Waiting to Exhale (1995)                  Comedy|Drama   
4         5  Father of the Bride Part II (1995)                        Comedy   

   movie_ID  
0         0  
1         1  
2         2  
3         3  
4         4  


,user_id,movie_id,rating,timestamp,user_ID,movie_ID
0,1,1193,5,978300760,0,1176
1,2,1193,5,978298413,1,1176
2,12,1193,4,978220179,11,1176
3,15,1193,4,978199279,14,1176
4,17,1193,5,978158471,16,1176


In [3]:
class MovieLens1m:
    def __init__(self,batch_size) :
        self.batch_size = batch_size
        self.items = pd.read_csv("./Data/movies.csv",encoding="latin-1",sep="\t")
        self.items["movie_ID"] = np.arange(stop=movies.shape[0]) 

        self.users = pd.read_csv("./Data/users.csv",encoding="latin-1",sep="\t")
        self.users["user_ID"] = np.arange(stop=users.shape[0]) 

        self.ratings = pd.read_csv("./Data/ratings.csv",encoding="latin-1",sep="\t")

        self.ratings = pd.merge(pd.merge(ratings,users[["user_id","user_ID"]],on="user_id"),movies[["movie_id","movie_ID"]],on="movie_id")
        

                

array([[   0, 1176],
       [   1, 1176],
       [  11, 1176],
       ...,
       [5779, 2776],
       [5850, 3538],
       [5937, 2840]])

In [29]:
class dataram:
    def __init__(self,batch_size,users) -> None:
          self.batch_size = batch_size
          self.exist_users = users
          self.n_users = len(self.exist_users)
    def sample(self):
        if self.batch_size <= self.n_users:
            users = rd.sample(self.exist_users.tolist(), self.batch_size)
        else:
            users = [
                rd.choice(self.exist_users) for _ in range(self.batch_size)
            ]
        return users
users = np.arange(100)
data = dataram(10,users)
n_batch = data.n_users//10+1
ex = []
for batch in range(n_batch):
    res=set(data.sample())
    print(res)
    ex.append(res)
    
    


{64, 2, 99, 68, 72, 77, 49, 50, 19, 54}
{0, 68, 11, 75, 24, 58, 59, 29, 30, 63}
{11, 76, 46, 81, 21, 54, 22, 88, 29, 95}
{64, 1, 35, 6, 38, 43, 54, 23, 86, 56}
{32, 71, 9, 81, 50, 19, 54, 56, 92, 62}
{0, 97, 98, 99, 35, 37, 52, 93, 62, 63}
{32, 68, 36, 40, 11, 13, 14, 81, 83, 86}
{98, 69, 70, 75, 13, 83, 54, 87, 57, 93}
{64, 65, 36, 42, 75, 46, 15, 80, 18, 90}
{33, 68, 8, 44, 45, 51, 19, 88, 27, 28}
{64, 3, 99, 76, 78, 82, 51, 84, 23, 93}


In [37]:
for i in range(n_batch):
    inters = dict()
    for j in range(i+1,n_batch):

        inter = ex[i].intersection(ex[j])
        if len(inter)!=0 :
            inters[j]=inter
    print(f"batch {i}:{ex[i]}|intersaction {inters}")

batch 0:{64, 2, 99, 68, 72, 77, 49, 50, 19, 54}|intersaction {1: {68}, 2: {54}, 3: {64, 54}, 4: {50, 19, 54}, 5: {99}, 6: {68}, 7: {54}, 8: {64}, 9: {19, 68}, 10: {64, 99}}
batch 1:{0, 68, 11, 75, 24, 58, 59, 29, 30, 63}|intersaction {2: {11, 29}, 5: {0, 63}, 6: {11, 68}, 7: {75}, 8: {75}, 9: {68}}
batch 2:{11, 76, 46, 81, 21, 54, 22, 88, 29, 95}|intersaction {3: {54}, 4: {81, 54}, 6: {81, 11}, 7: {54}, 8: {46}, 9: {88}, 10: {76}}
batch 3:{64, 1, 35, 6, 38, 43, 54, 23, 86, 56}|intersaction {4: {56, 54}, 5: {35}, 6: {86}, 7: {54}, 8: {64}, 10: {64, 23}}
batch 4:{32, 71, 9, 81, 50, 19, 54, 56, 92, 62}|intersaction {5: {62}, 6: {32, 81}, 7: {54}, 9: {19}}
batch 5:{0, 97, 98, 99, 35, 37, 52, 93, 62, 63}|intersaction {7: {98, 93}, 10: {99, 93}}
batch 6:{32, 68, 36, 40, 11, 13, 14, 81, 83, 86}|intersaction {7: {83, 13}, 8: {36}, 9: {68}}
batch 7:{98, 69, 70, 75, 13, 83, 54, 87, 57, 93}|intersaction {8: {75}, 10: {93}}
batch 8:{64, 65, 36, 42, 75, 46, 15, 80, 18, 90}|intersaction {10: {64}}
b